In [1]:
import tensorflow as tf

# 导入 tensorflow.mnist 数据集工具包
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 载入数据集
# 
# 下载 数据集
# 将标签转换为 one_hot 编码形式
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# tensorflow 版本更新,以前的数据集下载等 方法将不再支持,回警告！！！

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 创建 3 个占位符
# 
# 输入样本、输入标签
x = tf.placeholder(dtype=tf.float32, shape=(None, 784))
y = tf.placeholder(dtype=tf.float32, shape=(None, 10))

# dropout 参数设置
keep_prob = tf.placeholder(dtype=tf.float32)

In [4]:
# 创建神经网络
# 
# 输入层
weight_L1 = tf.Variable(tf.truncated_normal(shape=(784, 2000), mean=0, stddev=0.1))
biase_L1 = tf.Variable(tf.zeros(shape=(1, 2000)) + 0.1)
wx_plus_b_L1 = tf.matmul(x, weight_L1) + biase_L1

# 双曲正切函数作为激活函数
L1 = tf.nn.tanh(x=wx_plus_b_L1)

# Dropout 层
# 
# x 为该层神经元输出，keep_prob 为每层神经元工作比
L1_dropout = tf.nn.dropout(x=L1, keep_prob=keep_prob)

In [5]:
# 中间层
# 
weight_L2 = tf.Variable(tf.truncated_normal(shape=(2000, 2000), mean=0, stddev=0.1))
biase_L2 = tf.Variable(tf.zeros(shape=(1, 2000)) + 0.1)
wx_plus_b_L2 = tf.matmul(L1_dropout, weight_L2) + biase_L2

# 激活函数 tanh()
L2 = tf.nn.tanh(x=wx_plus_b_L2)

# Dropout 层
# 
L2_dropout = tf.nn.dropout(x=L2, keep_prob=keep_prob)

# # 创建输出层
# prediction = tf.nn.softmax(wx_plus_b_L2)

In [6]:
# 输出层
# 
weight_L3 = tf.Variable(tf.truncated_normal(shape=(2000, 10), mean=0, stddev=0.1))
biase_L3 = tf.Variable(tf.truncated_normal(shape=(1, 10), mean=0, stddev=0.1))
wx_plus_b_L3 = tf.matmul(L2_dropout, weight_L3) + biase_L3

# 激活函数 tanh()
prediction = tf.nn.softmax(wx_plus_b_L3)

In [7]:
# 损失函数、优化器
# 
# 定义损失函数
# loss = tf.reduce_mean(tf.square(y - prediction))
# 
# 定义损失函数 对数似然代价函数与 soft 组合
# 
# 训练时间、准确度有所提升
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))

# 创建优化器，优化损失函数
train = tf.train.GradientDescentOptimizer(learning_rate=0.2).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
# 判断准去率
# 
# tf.equal(x, y),判断 x, y 是否相等，返回 bool 类型 (tensor 里的值为 bool)
# tf.argmax() 查找最大值，返回索引值
# tensor 的轴(是从 0 开始的)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))

# tf.case() 将 bool 类型,转换为 tf.float 类型
# 
# 疑问：acc 为什么 会有 4 位小数，出现呢？(tensor(100, 10), 里面不 1 就是 0)，我怎么感觉顶天就是 2 位小数
# 解答：准确率是测试集，测试的(10000)张照片,固有 4 位小数出现
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 保存模型
saver = tf.train.Saver()

In [9]:
# 初始化图中变量
init_op = tf.global_variables_initializer()

In [10]:
# 创建会话
with tf.Session() as sess:
    sess.run(init_op)
   
    test_acc_1 = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
    
    # 载入模型
    saver.restore(sess=sess, save_path=tf.train.latest_checkpoint(checkpoint_dir='./net'))
    test_acc_2 = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})

    print('没有载入模型准确率: ' + str(test_acc_1) + '\n', '载入模型准确率: ' + str(test_acc_2))


INFO:tensorflow:Restoring parameters from ./net\my_model
没有载入模型准确率: 0.146
 载入模型准确率: 0.9433
